##  This Notebook will gather GoogleTrends Data by using Pytrends
(the data is availible in the directory so use that instead of this notebook)

In [13]:
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import datetime
import warnings


In [14]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
final_df = pd.read_csv("final.csv")
original_df = pd.read_stata("final.dta")

In [15]:
player_dict = dict(zip(original_df["odds_player"], original_df["player"]))

First Section looks at changing player names to names that google will recognise

In [16]:
final = final_df.replace({"player": player_dict})
final = final.replace({"opponent": player_dict})
final

Unnamed: 0  match_id                 player                  opponent  \
0           12773     12773        Casey Dellacqua           Elina Svitolina   
1           12774     12774          Camila Giorgi         Lara Arruabarrena   
2           12775     12775        Olga Govortsova              Alize Cornet   
3           12776     12776        Jelena Jankovic           Evgeniya Rodina   
4           12777     12777       Angelique Kerber  Anastasia Pavlyuchenkova   
...           ...       ...                    ...                       ...   
20927        4139      4139  Ekaterina Alexandrova              Kiki Bertens   
20928        4141      4141     Patricia Maria Tig             Magda Linette   
20929        4142      4142          Maria Sakkari            Elena Rybakina   
20930        4143      4143           Kiki Bertens            Elena Rybakina   
20931        4143      4143         Elena Rybakina              Kiki Bertens   

         year  month   day  rankdiff  rankdist  wikibuzz  inv_bet365  \
0      2015.0    7.0   2.0      44.0  0.042430  0.000000    0.546448   
1      2015.0    7.0   2.0     -53.0 -0.019485  0.000000    0.900901   
2      2015.0    7.0   2.0      95.0  0.028840  0.000000    0.476190   
3      2015.0    7.0   2.0     -71.0 -0.023432  0.000000    0.819672   
4      2015.0    7.0   2.0     -32.0 -0.076190  0.000000    0.862069   
...       ...    ...   ...       ...       ...       ...         ...   
20927  2020.0    2.0  15.0      20.0  0.089286  1.323982    0.500000   
20928  2020.0    2.0  15.0      63.0  0.014286 -0.517943    0.381679   
20929  2020.0    2.0  15.0      -4.0 -0.007619 -1.612937    0.421941   
20930  2020.0    2.0  16.0     -17.0 -0.085000 -2.935837    0.636943   
20931  2020.0    2.0  16.0      17.0  0.085000  2.935837    0.421941   

        inv_avg   inv_max  encoded_tournament  win  
0      0.537634  0.518135                 120    1  
1      0.892857  0.877193                 120    1  
2      0.465116  0.444444                 120    1  
3      0.813008  0.793651                 120    1  
4      0.877193  0.847458                 120    1  
...         ...       ...                 ...  ...  
20927  0.483092  0.454545                  70    0  
20928  0.396825  0.378788                 108    0  
20929  0.431035  0.408163                  70    0  
20930  0.641026  0.621118                  70    1  
20931  0.414938  0.386100                  70    0  

[20932 rows x 15 columns]

In [17]:
final.to_csv("match_df.csv")

## Player DF is used to find unique players so as to gather the GoogleTrend Monthly Data

In [6]:
player_df = final["player"]

In [7]:
x = player_df.drop_duplicates()
name_df = x.to_frame()
name_df

player
0       Casey Dellacqua
1         Camila Giorgi
2       Olga Govortsova
3       Jelena Jankovic
4      Angelique Kerber
...                 ...
20409     Hourigan P.M.
20674  Schmiedlova A.K.
20680    Fernandez L.A.
20859         Eikeri U.
20874      Muramatsu C.

[430 rows x 1 columns]

In [10]:
name_df = name_df.reset_index()
name_df #412 players, just need to append with the google trends data

index            player
0        0   Casey Dellacqua
1        1     Camila Giorgi
2        2   Olga Govortsova
3        3   Jelena Jankovic
4        4  Angelique Kerber
..     ...               ...
425  20409     Hourigan P.M.
426  20674  Schmiedlova A.K.
427  20680    Fernandez L.A.
428  20859         Eikeri U.
429  20874      Muramatsu C.

[430 rows x 2 columns]

## Google Trends
First install pytrends

In [5]:
pip install --upgrade --user git+https://github.com/GeneralMills/pytrends

  Cloning https://github.com/GeneralMills/pytrends to c:\users\griev\appdata\local\temp\pip-req-build-sszijzw0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pytrends: filename=pytrends-4.9.2-py3-none-any.whl size=15605 sha256=7c3003823c51c379bcc495bbd2f250b48448b59cf81b104b1e41c3b01bef066e
  Stored in directory: C:\Users\griev\AppData\Local\Temp\pip-ephem-wheel-cache-eulhkl7y\wheels\16\ff\41\df76076b615c3bc8c806e9f375bbccba902b4c66f0472d99b1
Successfully built pytrends
  Attempting uninstall: pytrends
    Found existing installation: pytrends 4.9.2
    Uninstalling pytrends-4.9.2:
      Successfully u

In [40]:
import pandas as pd
from pytrends.request import TrendReq

In [41]:
pytrend = TrendReq()

In [42]:
# give dates in and it return dates from a year before in a format pytrends can read
def to_date(year,month,day):
    prev_year = str(year-1)[:-2]
    year = str(year)[:-2]
    month = str(month)[:-2]
    if len(month) == 1:
        month = "0" + month
    day = str(day)[:-2]
    if len(day) == 1:
        day = "0" + day
    date = year+"-"+month+"-"+day
    prev_date = prev_year+"-"+month+"-"+day
    return(prev_date +" "+ date)

## Need 12 months before the match and the day before the match

In [44]:
import json

import pandas as pd
import requests

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests import status_codes

from pytrends import exceptions

from urllib.parse import quote


BASE_TRENDS_URL = 'https://trends.google.com/trends'


Elina Svitolina

In [45]:
import time
from pathlib import Path 

In [46]:
player_df

0              Casey Dellacqua
1                Camila Giorgi
2              Olga Govortsova
3              Jelena Jankovic
4             Angelique Kerber
                 ...          
20927    Ekaterina Alexandrova
20928       Patricia Maria Tig
20929            Maria Sakkari
20930             Kiki Bertens
20931           Elena Rybakina
Name: player, Length: 20932, dtype: object

In [210]:
#Gathers data for all of the players in the DF and writes it to the directory

kw_list = []
for i in player_df:
    print(i)
    ptr = TrendReq(hl='en-AU',retries=5, backoff_factor=0.1)
    kw_list = [i]
    ptr.build_payload(kw_list, cat=0, timeframe="2014-01-01 2020-01-01", geo='', gprop='')
    IOT = ptr.interest_over_time()
    filepath = Path('players 2014-2022/{}.csv'.format(i)) 
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    IOT.to_csv(filepath)
    time.sleep(60)

Kateryna Bondarenko
Mandy Minella
Arantxa Rus
Laura Siegemund
Maryna Zanevska
Sesil Karatantcheva
Zhang Shuai
Sally Peers
Richèl Hogenkamp
Kristina Kucova
Timea Babos
Ysaline Bonaventure
Serena Williams
Julia Goerges
Anna-Lena Friedsam
Sorana Cirstea
Teliana Pereira
Katerina Siniakova
Alizé Lim
Alexandra Dulgheru
Rebecca Peterson
Anna Tatishvili
Barbora Strycova
Andreea Mitu
Klara Koukalova
Danka Kovinic
Yulia Putintseva
Polona Hercog
Mona Barthel
Sara Errani
Yaroslava Shvedova
Daria Gavrilova
Yanina Wickmayer
Jelena Ostapenko
Dominika Cibulkova
Risa Ozaki
Tamira Paszek
Barbara Haas
Vitalia Diatchenko
Margarita Gasparyan
Anastasija Sevastova
Olga Savchuk
Aliaksandra Sasnovich
Lucie Hradecka
Bojana Jovanovski
Ipek Soylu
Patricia Mayr-Achleitner
Daniela Hantuchova
Alexandra Panova
Francesca Schiavone
Johanna Larsson
Tsvetana Pironkova
Roberta Vinci
Anna Karolina Schmiedlova
Kirsten Flipkens
Karin Knapp
Urszula Radwanska
Paula Ormaechea
Maria Fernanda Alves
Magda Linette
Carolina Alves
Va

In [45]:
# Use in conjuncture with later notebooks, if any data is missing return back here and the notebook will gather the data.
missing = pd.read_csv("Missing_players.csv")
kw_list = []
for i in missing["0"]:
    print(i)
    ptr = TrendReq(hl='en-AU',retries=5, backoff_factor=20)
    kw_list = [i]
    ptr.build_payload(kw_list, cat=0, timeframe="2014-01-01 2020-01-01", geo='', gprop='')
    IOT = ptr.interest_over_time()
    filepath = Path('players 2014-2022/{}.csv'.format(i)) 
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    IOT.to_csv(filepath)
    time.sleep(60)

Katharina Gerlach
Carina Witthoeft
Stephanie Vogt
Olga Fridman
Marina Melnikova
Barbora Krejcikova
Amra Sadikovic
Julia Glushko
Anna Zaja
Katharina Hobgarski
Nadia Podoroska
Fanny Stollar
Paula Ormaechea
Anna-Lena Friedsam
Cagla Buyukakcay
Nina Stojanovic
Jule Niemeier
Sabine Lisicki
Sorana Cirstea
Magda Linette
Pauline Parmentier
Saisai Zheng
Anastasia Pavlyuchenkova
Ana Bogdan
Evgeniya Rodina
Zarina Diyas
Vera Zvonareva
Luksika Kumkhum
Destanee Aiava
Kristina Mladenovic
Kiki Bertens
Elise Mertens
Kimberly Birrell
Daria Kasatkina
Eugenie Bouchard
Madison Brengle
Harriet Dart
Priscilla Hon
Lauren Davis
Lara Arruabarrena
Ons Jabeur
Zhang Shuai


KeyboardInterrupt: 

## Now I've got the Monthly Data, Want to extract the daily data for a month, per match

In [48]:
from pytrends.request import TrendReq
from pytrends import dailydata

def collect_trend_score(keyword):
    pytrend = TrendReq() 
    pytrend.build_payload(kw_list=[keyword])
    df = pytrend.interest_over_time()
    return df

def resample_trend_score_df(df, keyword):
    trends = df[keyword].resample('D', convention = 'start').pad()
    trends = pd.DataFrame(trends)
    trends.rename(columns = {keyword:'trend_score'}, inplace = True)
    return trends

In [16]:
from calendar import monthrange

(2, 31)

In [49]:
#For a given year/month gives the range of days in a format pytrends can read
def to_month(year,month):
    total_days = monthrange(year,month)[1]
    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)
    month_start = str(year) + "-" + str(str_month) + "-01"
    month_end = str(year) + "-" + str(str_month) + "-" + str(total_days)
    return(month_start +" "+ month_end)

In [40]:
failed = [] #this gathers daily data for each player (will take about a week)
for i in final["player"]:
    print(i)
    player_df = final.loc[final["player"]==i]
    player_df["year"] = player_df["year"].astype(int)
    player_df["month"] = player_df["month"].astype(int)
    df_ym = pd.DataFrame(player_df.loc[:,["year", "month"]].drop_duplicates().values)
    
    for year,month in zip(df_ym[0],df_ym[1]):
        print(year,month)
        ptr = TrendReq(hl='en-AU',retries=5, backoff_factor=0.1)
        kw_list = [i]
        ptr.build_payload(kw_list, cat=0, timeframe=to_month(year,month), geo='', gprop='')
        try:
            IOT = ptr.interest_over_time()
            filepath = Path('players/{}/{}_{}_{}.csv'.format(i,year,month,i)) 
            filepath.parent.mkdir(parents=True, exist_ok=True)  
            IOT.to_csv(filepath)
        except:
            print("Request Failed")
            failed.append(tuple([year,month,i]))
        time.sleep(31)

Ekaterina Alexandrova
2017 1
2017 2
2017 4
2017 6
2017 7
2017 8
2017 9
2018 1
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2019 1
2019 2
2019 3
2019 4
Bad...
2019 6
2019 7
2019 8
Bad...
2019 9
2019 10
Bad...
Aryna Sabalenka
2017 2
2017 8
2017 9
2017 10
2018 1
2018 3
2018 4
Bad...
2018 5
Bad...
2018 6
Bad...
2018 7
2018 8
2018 9
2018 10
2018 11
2019 1
2019 2
2019 3
Bad...
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
Bad...
2019 10
Tamara Korpatsch
2017 2
2017 4
2017 6
2017 7
2018 4
2018 5
2018 7
2019 4
2019 5
2019 7
2019 10
Renata Zarazua
2017 2
2017 4
2018 2
2018 4
2019 2
2019 4
2019 5
Fiona Ferro
2017 3
2017 4
Bad...
2017 5
2018 5
2018 9
Bad...
2018 10
2019 1
Bad...
2019 2
2019 4
2019 5
2019 6
2019 7
Bad...
2019 8
2019 9
2019 10
Kayla Day
2017 3
Bad...
2017 4
2017 8
2018 2
2018 3
Amanda Anisimova
2017 3
2017 5
2018 3
2018 8
2018 9
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
Kristie Ahn
2017 4
2017 6
Bad...
2017 8
2017 9
2018 1
2018 4
2018 6
2018 7
2018 9
Bad...
20

In [87]:
failed_df = pd.DataFrame(failed) #all failed requests are recorded and written to csv 
failed_df.to_csv("Missing_player_months.csv")

In [56]:
missing_players_df = pd.read_csv("Missing_player_months.csv")
missing_players_df

Unnamed: 0                          0     1  2
0             0            Petra Cetkovska  2015  8
1             1                  Mayo Hibi  2015  8
2             2     Lourdes Domínguez Lino  2015  8
3             3            Johanna Larsson  2015  8
4             4            Marina Erakovic  2015  8
..          ...                        ...   ... ..
199         395         Anastasia Potapova  2019  6
200         396              Jil Teichmann  2019  6
201         397  Anna Karolina Schmiedlova  2019  6
202         398                Kristie Ahn  2019  6
203         399           Yanina Wickmayer  2019  6

[204 rows x 4 columns]

In [57]:
#gather failed daily requests
failed = []
for player,year,month in zip(missing_players_df["0"],missing_players_df["1"],missing_players_df["2"]):
    print(year,month,player)
    ptr = TrendReq(hl='en-AU',retries=5, backoff_factor=0.1)
    kw_list = [player]
    ptr.build_payload(kw_list, cat=0, timeframe=to_month(year,month), geo='', gprop='')
    try:
        IOT = ptr.interest_over_time()
        filepath = Path('players/{}/{}_{}_{}.csv'.format(player,year,month,player)) 
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        IOT.to_csv(filepath)
    except:
        print("Request Failed")
        failed.append(tuple([year,month,player]))
    time.sleep(31)

2015 8 Petra Cetkovska
2015 8 Mayo Hibi
2015 8 Lourdes Domínguez Lino
2015 8 Johanna Larsson
2015 8 Marina Erakovic
2015 8 Alexandra Dulgheru
2015 8 Johanna Konta
2015 8 Kirsten Flipkens
2015 8 Aliaksandra Sasnovich
2015 8 Alexandra Panova
2015 8 Jelena Ostapenko
2015 8 Evgeniya Rodina
2015 8 Tereza Mrdeza
2015 8 Elena Vesnina
2015 8 Laura Robson
2015 8 Qiang Wang
2015 8 Maria Sakkari
2015 8 Francesca Schiavone
2015 8 Jamie Loeb
2016 2 Naomi Broady
2016 2 Nicole Gibbs
2016 2 Alexandra Dulgheru
2016 2 Risa Ozaki
2016 2 Marcela Zacarias
2016 2 Irina Falconi
2016 2 Zhaoxuan Yang
2016 4 Jelena Jankovic
2016 4 Margarita Gasparyan
2016 4 Coco Vandeweghe
2016 7 Irina-Camelia Begu
2016 7 Laura Pigossi
2016 7 Aleksandrina Naydenova
2016 7 Chang Kai-chen
2016 7 Zhang Yuxuan
2016 7 Montserrat Gonzalez
2016 7 Gabriela Ce
2016 7 Junri Namigata
2016 7 Emily Webley-Smith
2016 7 Lyudmyla Kichenok
2016 7 Nicha Lertpitaksinchai
2016 7 Peangtarn Plipuech
2016 7 Alizé Lim
2016 7 Laura Pous-Tio
2016 7 Nadi

## Data Should Be Gathered Now :) 
